In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import random
from utils import plot_samples_with_labels, classify, plot_conf_matrix

In [ ]:
# Set the seed for reproducibility
random.seed(42)

### Load and explore data

In [ ]:
# Download the MNIST dataset (handwritten digit images with labels)
mnist = keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
plot_samples_with_labels(training_images, training_labels, num_samples = 25, cmap = plt.cm.binary, randomize= False)

#### Data preparation

In [ ]:
training_images = training_images / 255.0
test_images = test_images / 255.0

# Reshape the data to include a channel dimension
training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

### Build the model

In [ ]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential([
    Flatten(input_shape=(28, 28, 1)),  # 1 = grayscale
    Dense(units=50, activation='relu', input_shape=(28, 28, 1)),
    Dense(units=50, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

### Train the model

In [ ]:
BATCH_SIZE = 32
num_training_images = training_images.shape[0]

history = model.fit(
    training_images, training_labels,
    epochs=10,
    batch_size=BATCH_SIZE,
    validation_data=(training_images, training_labels)
)

In [ ]:
model.save('models/digit_non_CNN.h5')

### Using the model to classify new images

In [ ]:
from tensorflow.keras.models import load_model

model_non_cnn = load_model('models/digit_non_CNN.h5')

In [ ]:
def classify(test_images, model):
    """
    Predicts the class labels for a batch of test images using a trained model.

    Args:
        test_images (numpy.ndarray): Preprocessed test images ready for prediction.
        model (tensorflow.keras.Model): A trained model to classify the images.

    Returns:
        numpy.ndarray: An array of predicted class labels.
    """
    predictions = model.predict(test_images)    # Predict probabilities
    predicted_class = np.argmax(predictions, axis=-1)    # Get class with highest probability
    return predicted_class


In [ ]:
predicted_labels = classify(test_images, model_non_cnn)
predicted_labels = np.array(predicted_labels)

In [ ]:
plot_samples_with_labels(test_images, test_labels, predicted_labels, num_samples = 10, randomize= True)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate metrics
print("Accuracy:", accuracy_score(test_labels, predicted_labels))
print("\nClassification Report:\n")
print(classification_report(test_labels, predicted_labels))

In [ ]:
# Generate confusion matrix
plot_conf_matrix(test_labels, predicted_labels)

In [ ]:
# Visualize some misclassified images

misclassified_indices = (test_labels != predicted_labels)
misclassified_images = test_images[misclassified_indices]
misclassified_true_labels = test_labels[misclassified_indices]
misclassified_predicted_labels = predicted_labels [misclassified_indices]

if len(misclassified_images) > 0:
    plot_samples_with_labels(misclassified_images, misclassified_true_labels, misclassified_predicted_labels, num_samples = 10, randomize= True)
else:
    print("No misclassified images found in the selected batch.")


Qué dificultades encuentra nuestro modelo?

Ideas para solucionarlas?


Este modelo tiene una arquitectura simple, utiliza únicamente capas densas y aplanadas (Fully Connected Layers) para procesar imágenes, lo que ignora la información espacial inherente a las imágenes (como bordes, patrones y texturas).

El modelo se entrena y valida con los mismos datos de entrenamiento, lo que provoca sobreajuste.
La precisión del modelo es alta en los datos de entrenamiento, pero podría no generalizar bien en datos no vistos.

No se han aplicado técnicas como Dropout o Batch Normalization, lo que hace que el modelo sea más propenso al sobreajuste.

Algunas imágenes del conjunto de datos pueden ser difíciles de clasificar debido a escritura poco clara o mal formada.

Este modelo tiene limitaciones al enfrentarse a problemas más complejos como imágenes a color o conjuntos de datos más grandes.

In [ ]:
# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Configurar semilla para reproducibilidad
np.random.seed(42)

# Cargar datos MNIST
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# Visualizar ejemplos del conjunto de datos
def plot_samples(images, labels, num_samples=25, cmap='gray'):
    plt.figure(figsize=(10, 10))
    for i in range(num_samples):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=cmap)
        plt.xlabel(labels[i])
    plt.show()

plot_samples(training_images, training_labels)

# **Preprocesamiento de los datos**
# Normalizar las imágenes al rango [0, 1]
training_images = training_images / 255.0
test_images = test_images / 255.0

# Añadir dimensión de canal (grayscale)
training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

# Separar un conjunto de validación del conjunto de entrenamiento
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    training_images, training_labels, test_size=0.2, random_state=42
)

# **Aumento de datos**
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train)

# **Construcción del modelo CNN**
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularización para evitar sobreajuste
    Dense(10, activation='softmax')  # 10 clases (0-9)
])

# Compilar el modelo
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Mostrar resumen del modelo
cnn_model.summary()

# **Entrenamiento del modelo**
history = cnn_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=10,
    steps_per_epoch=len(X_train) // 32
)

# **Evaluar el modelo en el conjunto de prueba**
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels)
print(f'\nPérdida en prueba: {test_loss:.4f}')
print(f'Precisión en prueba: {test_accuracy:.2%}')

# **Matriz de confusión y reporte de clasificación**
y_pred = np.argmax(cnn_model.predict(test_images), axis=-1)

print("\nClassification Report:")
print(classification_report(test_labels, y_pred))

print("Confusion Matrix:")
conf_matrix = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(8, 8))
plt.imshow(conf_matrix, cmap='Blues')
plt.title('Confusion Matrix')
plt.colorbar()
plt.show()

# **Visualizar ejemplos mal clasificados**
misclassified_indices = np.where(test_labels != y_pred)[0]
misclassified_images = test_images[misclassified_indices]
misclassified_true_labels = test_labels[misclassified_indices]
misclassified_predicted_labels = y_pred[misclassified_indices]

def plot_misclassified(images, true_labels, predicted_labels, num_samples=10):
    plt.figure(figsize=(10, 10))
    for i in range(min(num_samples, len(images))):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i].reshape(28, 28), cmap='gray')
        plt.xlabel(f'True: {true_labels[i]}, Pred: {predicted_labels[i]}')
    plt.show()

if len(misclassified_images) > 0:
    plot_misclassified(misclassified_images, misclassified_true_labels, misclassified_predicted_labels)
else:
    print("No misclassified images found.")
